# CruncherLab #

## Import modules and set up the environment ##

In [1]:
import getopt
import os
import sys
import time

from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

from yanux.cruncher.model.loader import JsonLoader
from yanux.cruncher.model.wifi import WifiLogs
from yanux.cruncher.ml.experiments import *

pd.set_option('display.max_rows', 10000)

## Initialize Input & Output Data Directories and other parameters ##

In [2]:
input_data_directory = "data"
output_data_directory = "out"

print("Input Data Directory is:", input_data_directory)
print("Output Data Directory is", output_data_directory)

Input Data Directory is: data
Output Data Directory is out


### Create the output directory if it doesn't exist ###

In [3]:
if not os.path.exists(output_data_directory):
    os.makedirs(output_data_directory)

## Load Data from the Input Data Directory ##

In [4]:
json_loader = JsonLoader(input_data_directory)
wifi_logs = WifiLogs(json_loader.json_data)
#wifi_logs.shuffle_samples()

Store the data into a Pandas Dataframe, in which each Wi-Fi result reading is represented by a single line

In [5]:
wifi_results_columns = ["filename", "x", "y", "floor", "orientation", "sample_id", "mac_address",
                        "timestamp", "signal_strength"]

wifi_results = pd.DataFrame(wifi_logs.wifi_results(), columns=wifi_results_columns)
wifi_results.to_csv(output_data_directory + "/wifi_results.csv")

Identify the unique MAC Addresses present in the recorded data. Each one represents a single Wi-Fi Access Point.

In [6]:
mac_addresses = wifi_results.mac_address.unique()

Similarly, store the data into a Pandas Dataframe in which each line represents a single sampling cycle with *n* different readings for each of the Access Points within range. Those readings are stored as columns along each sample.

In [7]:
wifi_samples_columns = ["filename", "x", "y", "floor", "orientation", "sample_id", "timestamp"]
wifi_samples_columns.extend(mac_addresses)

wifi_samples = pd.DataFrame(wifi_logs.wifi_samples(), columns=wifi_samples_columns)
wifi_samples = wifi_samples.sort_values(["filename", "x", "y", "floor", "sample_id"]).reset_index(drop=True)
wifi_samples.to_csv(output_data_directory + "/wifi_samples.csv")

## Analyzing the Data Set ##

# *** TODO *** #

## Generate Train and Test Scenario ##

Initialize the train and test scenario generation parameters

In [8]:
raw = True
groupby_mean = False
groupby_max = False
groupby_min = False
data_partials = [0.35]
test_data_partials = [0.35]
filename_prefixes = ["point", "altPoint"]
subset_locations_values = [0.5]

In [9]:
print("Generating Training and Test Scenarios...")

data_scenarios = {}
test_data_scenarios = {}

full_data_scenarios = {}
prepare_full_data_scenarios(wifi_samples, full_data_scenarios,
                            raw=raw,
                            groupby_mean=groupby_mean,
                            groupby_max=groupby_max,
                            groupby_min=groupby_min)

full_test_data_scenarios = {}
prepare_full_data_scenarios(wifi_samples, full_test_data_scenarios,
                            raw=raw,
                            groupby_mean=groupby_mean,
                            groupby_max=groupby_max,
                            groupby_min=groupby_min)

data_scenarios.update(full_data_scenarios)
test_data_scenarios.update(full_test_data_scenarios)

partial_data_scenarios = {}
prepare_partial_data_scenarios(wifi_samples, partial_data_scenarios,
                               slice_at_the_end=False,
                               raw=raw,
                               groupby_mean=groupby_mean,
                               groupby_max=groupby_max,
                               groupby_min=groupby_min,
                               partials=data_partials)
partial_test_data_scenarios = {}
prepare_partial_data_scenarios(wifi_samples, partial_test_data_scenarios,
                               slice_at_the_end=True,
                               raw=raw,
                               groupby_mean=groupby_mean,
                               groupby_max=groupby_max,
                               groupby_min=groupby_min,
                               partials=test_data_partials)

data_scenarios.update(partial_data_scenarios)
test_data_scenarios.update(partial_test_data_scenarios)

filename_startswith_data_scenarios = {}
for filename_prefix in filename_prefixes:
    prepare_filename_startswith_data_scenarios(wifi_samples, filename_startswith_data_scenarios,
                                               raw=raw,
                                               groupby_mean=groupby_mean,
                                               groupby_max=groupby_max,
                                               groupby_min=groupby_min,
                                               filename_startswith=filename_prefix)
filename_startswith_test_data_scenarios = {}
for filename_prefix in filename_prefixes:
    prepare_filename_startswith_data_scenarios(wifi_samples, filename_startswith_test_data_scenarios,
                                               raw=raw,
                                               groupby_mean=groupby_mean,
                                               groupby_max=groupby_max,
                                               groupby_min=groupby_min,
                                               filename_startswith=filename_prefix)

data_scenarios.update(filename_startswith_data_scenarios)
test_data_scenarios.update(filename_startswith_test_data_scenarios)

subset_locations_data_scenarios = {}
for subset_locations in subset_locations_values:
    prepare_full_data_scenarios(subset_wifi_samples_locations(wifi_samples, subset_locations),
                                subset_locations_data_scenarios,
                                raw=raw,
                                groupby_mean=groupby_mean,
                                groupby_max=groupby_max,
                                groupby_min=groupby_min,
                                scenarios_suffix="subset_locations=" + str(subset_locations))

data_scenarios.update(subset_locations_data_scenarios)


path_direction_aggregated_data_scenarios = {}
prepare_path_direction_aggregated_data_scenarios(wifi_samples, path_direction_aggregated_data_scenarios,
                                         groupby_mean=groupby_mean,
                                         groupby_max=groupby_max,
                                         groupby_min=groupby_min)

data_scenarios.update(path_direction_aggregated_data_scenarios)

#Data Scenarios using sklearn's train_test_split
loc_coords = ["x", "y", "floor"]
X = wifi_samples[mac_addresses].copy()
y = wifi_samples[loc_coords].copy()
y["label"] = "(" + y.x.map(str) + ", " + y.y.map(str) + ", " + y.floor.map(str) + ")"

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y[loc_coords],
                                                    test_size=0.5,
                                                    stratify=y["label"].values,
                                                    random_state=0)
#X_train = X_train.reset_index(drop=True)
#X_test = X_test.reset_index(drop=True)
#y_train = y_train.reset_index(drop=True)
#y_test = y_test.reset_index(drop=True)

if raw or groupby_mean or groupby_max or groupby_min:
    #train data
    train_split_data = pd.concat([y_train, X_train], axis=1).reset_index(drop=True)
    #test data
    test_split_data = pd.concat([y_test, X_test], axis=1).reset_index(drop=True)
    if raw:
        #train data
        data_scenarios["train_test_split_data"] = train_split_data
        #test data
        test_data_scenarios["train_test_split_data"] = test_split_data
    if groupby_mean or groupby_max or groupby_min:
        #train data
        train_split_groupby_data = train_split_data.groupby(loc_coords, as_index=False)
        #test data
        test_split_groupby_data = test_split_data.groupby(loc_coords, as_index=False)
        if groupby_mean:
            #train data
            train_split_groupby_mean_data = train_split_groupby_data.mean()
            data_scenarios["train_test_split_mean_data"] = train_split_groupby_mean_data
            #test data
            test_split_groupby_mean_data = test_split_groupby_data.mean()
            data_scenarios["train_test_split_mean_data"] = test_split_groupby_mean_data
        if groupby_max:
            #train data
            train_split_groupby_max_data = train_split_groupby_data.max()
            data_scenarios["train_test_split_max_data"] = train_split_groupby_max_data
            #test data
            test_split_groupby_max_data = test_split_groupby_data.max()
            data_scenarios["train_test_split_max_data"] = test_split_groupby_max_data
        if groupby_min:
            #train data
            train_split_groupby_min_data = train_split_groupby_data.min()
            data_scenarios["train_test_split_min_data"] = train_split_groupby_min_data
            #test data
            test_split_groupby_min_data = test_split_groupby_data.min()
            data_scenarios["train_test_split_min_data"] = test_split_groupby_min_data

save_scenarios(data_scenarios, output_directory=output_data_directory, prefix="train_")
print("# Data Scenarios: " + str(len(data_scenarios)))
save_scenarios(test_data_scenarios, output_directory=output_data_directory, prefix="test_")
print("# Test Data Scenarios: " + str(len(test_data_scenarios)))
print("Scenarios Generated!")

Generating Training and Test Scenarios...
# Data Scenarios: 6
# Test Data Scenarios: 5
Scenarios Generated!


Set a train and test scenario to be used by default when testing.

In [10]:
default_data_scenario = data_scenarios["train_test_split_data"]
default_test_data_scenario = test_data_scenarios["train_test_split_data"]

## Playground ##

### Algorithm ###
Test if the different nearest neighbor search algorithms produce different results.

In [11]:
n_neighbors=5
weights="uniform"
metric="euclidean"
nan_filler=-100
algorithms = ["brute", "kd_tree", "ball_tree"]
leaf_sizes = range(10, 60, 10) 

curr_data = default_data_scenario.fillna(nan_filler)
curr_test_data = default_test_data_scenario.fillna(nan_filler)

# Just a metrics accumulator
metrics = []
for a in algorithms:
    for l in leaf_sizes:
        start_time = time.clock()
        results = knn_experiment(curr_data,
                                 mac_addresses,
                                 ["x", "y"],
                                 algorithm=a,
                                 leaf_size=l,
                                 n_neighbors=n_neighbors,
                                 weights=weights,
                                 metric=metric,
                                 test_data=curr_test_data)
        end_time = time.clock()
        curr_metrics = experiment_metrics(results)
        curr_metrics["algorithm"] = a
        curr_metrics["leaf_size"] = l
        curr_metrics["elapsed_time"] = end_time - start_time
        metrics.append(curr_metrics)

cols = ["algorithm", "leaf_size", "elapsed_time"] + list(curr_metrics.keys())[:-3]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-algorithm.csv")
metrics_table.sort_values(cols[3:])
#metrics_table.sort_values("elapsed_time")

,algorithm,leaf_size,elapsed_time,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
12,ball_tree,30,0.285012,2.344671,1.350736,7.31832,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
13,ball_tree,40,0.287389,2.344671,1.350736,7.31832,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
14,ball_tree,50,0.286672,2.344671,1.350736,7.31832,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
0,brute,10,3.508350,2.345659,1.349728,7.32024,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
1,brute,20,0.296914,2.345659,1.349728,7.32024,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
2,brute,30,0.352346,2.345659,1.349728,7.32024,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
3,brute,40,0.308374,2.345659,1.349728,7.32024,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
4,brute,50,0.328718,2.345659,1.349728,7.32024,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
5,kd_tree,10,0.308156,2.346271,1.349851,7.32344,1.581139,2.0,2.8,4.04,4.8,0.0,8.0
6,kd_tree,20,0.289095,2.346271,1.349851,7.32344,1.581139,2.0,2.8,4.04,4.8,0.0,8.0


### # Neighbors ###
Test how the *k* value influences performance metrics

In [12]:
n_neighbors=range(1,11,2)
weights="uniform"
metric="euclidean"
nan_filler=-100

curr_data = default_data_scenario.fillna(nan_filler)
curr_test_data = default_test_data_scenario.fillna(nan_filler)

# Just a metrics accumulator
metrics = []
for k in n_neighbors:
    curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                     mac_addresses,
                                                     ["x", "y"],
                                                     algorithm="brute",
                                                     n_neighbors=k,
                                                     weights=weights,
                                                     metric=metric,
                                                     test_data=curr_test_data))
    curr_metrics["k"] = k
    metrics.append(curr_metrics)

cols = ["k"] + list(curr_metrics.keys())[:-1]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-n_neighbors.csv")
metrics_table.sort_values(cols[1:])

,k,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
4,9,2.262385,1.379740,7.018259,1.333333,2.0,3.111111,4.00,4.444444,0.000000,8.444444
3,7,2.292893,1.363554,7.112918,1.428571,2.0,3.142857,4.00,4.857143,0.000000,8.571429
2,5,2.345659,1.349728,7.320240,1.581139,2.0,2.800000,4.04,4.800000,0.000000,8.000000
1,3,2.422055,1.349429,7.683667,2.000000,2.0,3.333333,4.00,4.666667,0.000000,7.333333
0,1,2.694031,1.430136,9.299000,2.000000,2.0,4.000000,4.00,6.000000,1.581139,8.000000


### Weights ###
Check whether the neighbors should have the same (*uniform*) or a weighted (*distance*-based) influence in the regression result.

In [13]:
n_neighbors=range(2,6,1)
weights=["uniform", "distance"]
metric="euclidean"
nan_filler=-100

curr_data = default_data_scenario.fillna(nan_filler)
curr_test_data = default_test_data_scenario.fillna(nan_filler)

# Just a metrics accumulator
metrics = []
for k in n_neighbors:
    for w in weights:
        curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                         mac_addresses,
                                                         ["x", "y"],
                                                         algorithm="brute",
                                                         n_neighbors=k,
                                                         weights=w,
                                                         metric=metric,
                                                         test_data=curr_test_data))
        curr_metrics["k"] = k
        curr_metrics["weights"] = w
        metrics.append(curr_metrics)

cols = ["k","weights"] + list(curr_metrics.keys())[:-2]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-weights.csv")
metrics_table.sort_values(cols[2:])

,k,weights,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
6,5,uniform,2.345659,1.349728,7.320240,1.581139,2.0,2.800000,4.040000,4.800000,0.000000,8.000000
7,5,distance,2.351124,1.339355,7.318071,1.581139,2.0,2.954719,4.125046,4.784092,0.002477,7.963439
4,4,uniform,2.379284,1.345526,7.467812,1.581139,2.0,3.000000,4.000000,5.000000,0.000000,7.500000
5,4,distance,2.380650,1.340794,7.461626,1.581139,2.0,3.064506,4.000000,4.941469,0.005376,7.505343
2,3,uniform,2.422055,1.349429,7.683667,2.000000,2.0,3.333333,4.000000,4.666667,0.000000,7.333333
3,3,distance,2.422847,1.343001,7.670232,1.897785,2.0,3.328610,4.000000,4.727789,0.000996,7.347844
0,2,uniform,2.508537,1.383770,8.203750,2.000000,2.0,3.000000,4.000000,5.000000,0.000000,8.000000
1,2,distance,2.511764,1.367973,8.176569,2.000000,2.0,3.072435,4.000000,4.968958,0.000000,8.000000


### Metric ###
Just test a few different distance metrics to assess if there is a better alternative than the plain old *euclidean* distance. The tested metrics include:
- Euclidean Distance
    - sqrt(sum((x - y)^2))
- Manhattan Distance
    - sum(|x - y|) 
- Chebyshev Distance
    - sum(max(|x - y|))
- Hamming Distance
    - N_unequal(x, y) / N_tot
- Canberra Distance
    - sum(|x - y| / (|x| + |y|))
- Braycurtis Similarity
    - sum(|x - y|) / (sum(|x|) + sum(|y|))
- S Euclidean Distance
    - sqrt(sum((x - y)^2 / V))
- Mahalanobis Distance
    - sqrt((x - y)' V^-1 (x - y))

The possible arguments are the following:
- p = The order of the norm of the difference
- V = array_like symmetric positive-definite covariance matrix.
- w = (N,) array_like weight vector.

In [14]:
n_neighbors=3
weights="uniform"
metric=[
        "euclidean","manhattan", "chebyshev",
        "hamming", "canberra", "braycurtis",
        #"seuclidean", "mahalanobis"
       ]
nan_filler=-100

curr_data = default_data_scenario.fillna(nan_filler)
curr_test_data = default_test_data_scenario.fillna(nan_filler)

# Just a metrics accumulator
metrics = []
for m in metric:
    if metric in ["mahalanobis", "seuclidean"]:
        metric_params = {'V': np.cov(curr_data[mac_addresses])}
    else:
        metric_params = None
    curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                     mac_addresses,
                                                     ["x", "y"],
                                                     algorithm="brute",
                                                     n_neighbors=n_neighbors,
                                                     weights=weights,
                                                     metric=m,
                                                     metric_params=metric_params,
                                                     test_data=curr_test_data))
    curr_metrics["metric"] = m
    metrics.append(curr_metrics)

cols = ["metric"] + list(curr_metrics.keys())[:-1]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-metric.csv")
metrics_table.sort_values(cols[1:])

,metric,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
1,manhattan,2.342303,1.322752,7.232556,1.581139,2.000000,3.333333,4.000000,4.666667,0.0,8.000000
5,braycurtis,2.360393,1.319523,7.309111,1.809498,2.000000,3.333333,4.000000,4.666667,0.0,8.000000
4,canberra,2.367128,1.340884,7.397667,2.000000,2.000000,3.333333,4.000000,4.666667,0.0,8.666667
0,euclidean,2.422055,1.349429,7.683667,2.000000,2.000000,3.333333,4.000000,4.666667,0.0,7.333333
2,chebyshev,2.673967,1.673636,9.945556,2.000000,2.000000,3.333333,4.720683,6.000000,0.0,12.666667
3,hamming,3.101303,2.428956,15.506111,1.333333,2.666667,4.000000,6.000000,8.365760,0.0,14.666667


### NaN filler values ###

Test which is the signal strength value that should be considered for Access Points that are currently out of range. This is needed as part of the process of computing the distance/similarity between different fingerprints.

In [15]:
n_neighbors=3
weights="uniform"
metric="euclidean"
nan_filler = [-1000000, -100, 0, 100, 1000000,
              data_scenarios["full_data"][mac_addresses].min().min()-1] 

# Just a metrics accumulator
metrics = []
for nf in nan_filler:
    curr_data = default_data_scenario.fillna(nf)
    curr_test_data = default_test_data_scenario.fillna(nf)
    curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                     mac_addresses,
                                                     ["x", "y"],
                                                     algorithm="brute",
                                                     n_neighbors=n_neighbors,
                                                     weights=weights,
                                                     metric=metric,
                                                     test_data=curr_test_data))
    curr_metrics["nan_filler"] = nf
    metrics.append(curr_metrics)

cols = ["nan_filler"] + list(curr_metrics.keys())[:-1]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-nan_filler.csv")
metrics_table.sort_values(cols[1:])

,nan_filler,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
5,-93.0,2.349402,1.280203,7.155333,2.0,2.000000,2.666667,4.000000,4.666667,0.0,8.666667
1,-100.0,2.422055,1.349429,7.683667,2.0,2.000000,3.333333,4.000000,4.666667,0.0,7.333333
0,-1000000.0,2.792590,1.606391,10.373889,2.0,2.586108,4.000000,4.666667,6.000000,0.0,9.261629
2,0.0,2.821181,1.693815,10.822333,2.0,2.527530,4.000000,5.333333,6.000000,0.0,10.000000
3,100.0,2.840647,1.693061,10.930000,2.0,2.666667,4.000000,5.333333,6.000000,0.0,12.000000
4,1000000.0,2.887696,1.759723,11.429222,2.0,2.666667,4.000000,5.333333,6.000000,0.0,12.000000


### Units ###
- dBm
- mW

In [16]:
n_neighbors=3
weights="uniform"
metric="euclidean"
nan_filler=-100

# Just a metrics accumulator
metrics = []

# Use the directly measured dBm values
curr_data = default_data_scenario.fillna(nan_filler)
curr_test_data = default_test_data_scenario.fillna(nan_filler)
curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                 mac_addresses,
                                                 ["x", "y"],
                                                 algorithm="brute",
                                                 n_neighbors=n_neighbors,
                                                 weights=weights,
                                                 metric=metric,
                                                 test_data=curr_test_data))
curr_metrics["units"] = "dBm"
metrics.append(curr_metrics)

# Convert to mW
curr_data[mac_addresses] = convert_to_units(curr_data[mac_addresses], from_units="dBm", to_units="mW")
curr_test_data[mac_addresses] = convert_to_units(curr_test_data[mac_addresses], from_units="dBm", to_units="mW")
curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                 mac_addresses,
                                                 ["x", "y"],
                                                 algorithm="brute",
                                                 n_neighbors=n_neighbors,
                                                 weights=weights,
                                                 metric=metric,
                                                 test_data=curr_test_data))
curr_metrics["units"] = "mW"
metrics.append(curr_metrics)

    
cols = ["units"] + list(curr_metrics.keys())[:-1]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-units.csv")
metrics_table.sort_values(cols[1:])

,units,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
0,dBm,2.422055,1.349429,7.683667,2.0,2.000000,3.333333,4.000000,4.666667,0.0,7.333333
1,mW,4.391064,3.228754,29.685444,2.0,3.333333,6.000000,9.333333,10.033333,0.0,24.041631


### Scaler ###
Test different data scaling and normalization approaches to find out if any of them provides a clear advantage over the others.

In [17]:
n_neighbors=3
weights="uniform"
metric="euclidean"
nan_filler = -100

scaler_values = {
                    "None": None,
                    "MinMaxScaler": preprocessing.MinMaxScaler(),
                    "StandardScaler": preprocessing.StandardScaler(),
                    "RobustScaler": preprocessing.RobustScaler(),
                    "NormalizerEuclidean": preprocessing.Normalizer(norm="l2"),
                    "NormalizerManhattan": preprocessing.Normalizer(norm="l1")
                }

# Just a metrics accumulator
metrics = []
for scaler_name, scaler in scaler_values.items():
    curr_data = default_data_scenario.fillna(nan_filler)
    curr_test_data = default_test_data_scenario.fillna(nan_filler)
    if scaler is not None:
        scaler.fit(curr_data[mac_addresses])
        curr_data[mac_addresses] = pd.DataFrame(scaler.transform(curr_data[mac_addresses]),
                                                columns=mac_addresses)
        curr_test_data[mac_addresses] = pd.DataFrame(scaler.transform(curr_test_data[mac_addresses]),
                                                     columns=mac_addresses)
    curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                     mac_addresses,
                                                     ["x", "y"],
                                                     algorithm="brute",
                                                     n_neighbors=n_neighbors,
                                                     weights=weights,
                                                     metric=metric,
                                                     metric_params=metric_params,
                                                     test_data=curr_test_data))
    curr_metrics["scaler"] = scaler_name
    metrics.append(curr_metrics)

cols = ["scaler"] + list(curr_metrics.keys())[:-1]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-scaler.csv")
metrics_table.sort_values(cols[1:])

,scaler,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
3,MinMaxScaler,2.329135,1.288369,7.081444,1.581139,2.0,2.666667,4.0,4.666667,0.0,7.333333
4,StandardScaler,2.369978,1.329271,7.380222,1.581139,2.0,3.333333,4.0,4.666667,0.0,7.333333
5,RobustScaler,2.404602,1.507054,8.048778,1.581139,2.0,3.333333,4.0,5.333333,0.0,8.666667
1,NormalizerEuclidean,2.410360,1.343461,7.611111,2.000000,2.0,3.333333,4.0,4.666667,0.0,8.000000
0,NormalizerManhattan,2.414360,1.349179,7.645778,2.000000,2.0,3.333333,4.0,4.666667,0.0,8.000000
2,None,2.422055,1.349429,7.683667,2.000000,2.0,3.333333,4.0,4.666667,0.0,7.333333


### Different Data Processing and Aggregation Scenarios ###

Testing different ways of processing offline and online data.

#### How does the amount of retrieved online data before affects positioning performance while considering the full offline data? And also, how do different agrregation strategies (i.e., mean, maximum and minimum signal strength per location) affect the results? ####

In [18]:
n_neighbors=3
weights="uniform"
metric="euclidean"
nan_filler=-100

curr_data_scenarios = {}
curr_data_scenarios.update(full_data_scenarios)

curr_test_data_scenarios = {}
curr_test_data_scenarios.update(full_test_data_scenarios)
curr_test_data_scenarios.update(partial_test_data_scenarios)

# Just a metrics accumulator
metrics = []
for data_scenario_name, data_scenario in curr_data_scenarios.items():
    for test_data_scenario_name, test_data_scenario in curr_test_data_scenarios.items():
        curr_data = data_scenario.fillna(nan_filler)
        curr_test_data = test_data_scenario.fillna(nan_filler)
        curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                         mac_addresses,
                                                         ["x", "y"],
                                                         algorithm="brute",
                                                         n_neighbors=n_neighbors,
                                                         weights=weights,
                                                         metric=metric,
                                                         test_data=curr_test_data))
        curr_metrics["data_scenario"] = data_scenario_name
        curr_metrics["test_data_scenario"] = test_data_scenario_name
        metrics.append(curr_metrics)

cols = ["data_scenario", "test_data_scenario"] + list(curr_metrics.keys())[:-2]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-data_scenarios.csv")
metrics_table.sort_values(cols[2:])

,data_scenario,test_data_scenario,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
1,full_data,partial_data_fraction=0.35,2.360641,1.391888,7.504444,2.0,2.0,2.666667,4.0,5.333333,0.0,9.333333
0,full_data,full_data,2.400288,1.318256,7.497444,2.0,2.0,3.333333,4.0,4.666667,0.0,9.333333


#### How does a varying number of samples per location on the offline data affects positioning performance? ####

In [19]:
n_neighbors=3
weights="uniform"
metric="euclidean"
nan_filler=-100

curr_data_scenarios = {}
curr_data_scenarios.update(full_data_scenarios)
curr_data_scenarios.update(partial_data_scenarios)

curr_test_data_scenarios = {}
curr_test_data_scenarios.update(full_test_data_scenarios)

# Just a metrics accumulator
metrics = []
for data_scenario_name, data_scenario in curr_data_scenarios.items():
    for test_data_scenario_name, test_data_scenario in curr_test_data_scenarios.items():
        curr_data = data_scenario.fillna(nan_filler)
        curr_test_data = test_data_scenario.fillna(nan_filler)
        curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                         mac_addresses,
                                                         ["x", "y"],
                                                         algorithm="brute",
                                                         n_neighbors=n_neighbors,
                                                         weights=weights,
                                                         metric=metric,
                                                         test_data=curr_test_data))
        curr_metrics["data_scenario"] = data_scenario_name
        curr_metrics["test_data_scenario"] = test_data_scenario_name
        metrics.append(curr_metrics)

cols = ["data_scenario", "test_data_scenario"] + list(curr_metrics.keys())[:-2]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-data_scenarios.csv")
metrics_table.sort_values(cols[2:])

,data_scenario,test_data_scenario,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
0,full_data,full_data,2.400288,1.318256,7.497444,2.0,2.0,3.333333,4.0,4.666667,0.0,9.333333
1,partial_data_fraction=0.35,full_data,2.530294,1.459231,8.529611,2.0,2.0,3.333333,4.0,6.000000,0.0,8.666667


#### Does orientation affect the positioning performance?  ####
Comparison between data collected when moving through the floor plan from left to right, top to bottom, and when moving in the opposite directions. 

In [20]:
n_neighbors=3
weights="uniform"
metric="euclidean"
nan_filler=-100

curr_data_scenarios = {}
curr_data_scenarios.update(filename_startswith_data_scenarios)

curr_test_data_scenarios = {}
curr_test_data_scenarios.update(filename_startswith_test_data_scenarios)

# Just a metrics accumulator
metrics = []
for data_scenario_name, data_scenario in curr_data_scenarios.items():
    for test_data_scenario_name, test_data_scenario in curr_test_data_scenarios.items():
        curr_data = data_scenario.fillna(nan_filler)
        curr_test_data = test_data_scenario.fillna(nan_filler)
        curr_metrics = experiment_metrics(knn_experiment(curr_data,
                                                         mac_addresses,
                                                         ["x", "y"],
                                                         algorithm="brute",
                                                         n_neighbors=n_neighbors,
                                                         weights=weights,
                                                         metric=metric,
                                                         test_data=curr_test_data))
        curr_metrics["data_scenario"] = data_scenario_name
        curr_metrics["test_data_scenario"] = test_data_scenario_name
        metrics.append(curr_metrics)

cols = ["data_scenario", "test_data_scenario"] + list(curr_metrics.keys())[:-2]
metrics_table = pd.DataFrame(metrics, columns=cols)
metrics_table.to_csv(output_data_directory + "/metrics-data_scenarios.csv")
metrics_table.sort_values(cols[2:])

,data_scenario,test_data_scenario,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
0,filename_startswith_data_point,filename_startswith_data_point,2.203274,1.201214,6.294444,2.0,2.0,2.666667,4.000000,4.666667,0.0,8.000000
3,filename_startswith_data_altPoint,filename_startswith_data_altPoint,2.462262,1.420160,8.075556,2.0,2.0,3.333333,4.000000,6.000000,0.0,8.000000
2,filename_startswith_data_altPoint,filename_startswith_data_point,2.602709,1.545212,9.157000,2.0,2.0,3.333333,4.666667,6.000000,0.0,9.333333
1,filename_startswith_data_point,filename_startswith_data_altPoint,2.707180,1.527109,9.656222,2.0,2.0,4.000000,4.666667,6.000000,0.0,8.000000


### Grid Search - Searching for estimator parameters ###

In [21]:
k_neighbors_values = range(1,11)
weights_values = [
                    "uniform",
                    "distance"
                 ]
metric_values = [
                    "euclidean",
                    "manhattan",
                    "chebyshev",
                    "hamming",
                    "canberra", 
                    "braycurtis"
                ]
algorithm_values = [
                    "auto",
                    "brute",
                    "ball_tree",
                    #"kd_tree"
                   ]
leaf_size_values = range(10, 60, 10)
nan_filler = -100

param_grid = {
                "n_neighbors": list(k_neighbors_values),
                "weights": weights_values,
                "metric": metric_values,
                "algorithm": algorithm_values,
                "leaf_size": list(leaf_size_values)
              }
knn = KNeighborsRegressor()
grid = GridSearchCV(knn, param_grid=param_grid, cv=10, n_jobs=-1, error_score=0,
                    scoring=sklearn.metrics.make_scorer(sklearn.metrics.r2_score,
                                                        greater_is_better=True,
                                                        needs_proba=False,
                                                        needs_threshold=False,
                                                        multioutput="uniform_average"))
grid.fit(X_train.fillna(nan_filler), y_train[["x", "y"]])
print("Best parameters set found on development set:")
print(grid.best_params_)
print("Grid scores on development set:")
for params, mean_score, scores in grid.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r" %(mean_score, scores.std() * 2, params))

Best parameters set found on development set:
{'leaf_size': 10, 'metric': 'manhattan', 'algorithm': 'auto', 'weights': 'distance', 'n_neighbors': 2}
Grid scores on development set:
0.994 (+/-0.008) for {'leaf_size': 10, 'metric': 'euclidean', 'algorithm': 'auto', 'weights': 'uniform', 'n_neighbors': 1}
0.994 (+/-0.008) for {'leaf_size': 10, 'metric': 'euclidean', 'algorithm': 'auto', 'weights': 'distance', 'n_neighbors': 1}
0.996 (+/-0.005) for {'leaf_size': 10, 'metric': 'euclidean', 'algorithm': 'auto', 'n_neighbors': 2, 'weights': 'uniform'}
0.996 (+/-0.005) for {'leaf_size': 10, 'metric': 'euclidean', 'algorithm': 'auto', 'weights': 'distance', 'n_neighbors': 2}
0.995 (+/-0.004) for {'leaf_size': 10, 'metric': 'euclidean', 'algorithm': 'auto', 'weights': 'uniform', 'n_neighbors': 3}
0.995 (+/-0.004) for {'leaf_size': 10, 'metric': 'euclidean', 'algorithm': 'auto', 'weights': 'distance', 'n_neighbors': 3}
0.994 (+/-0.004) for {'leaf_size': 10, 'metric': 'euclidean', 'algorithm': 'au

In [22]:
y_true, y_pred = y_test, grid.predict(X_test.fillna(-100))
#Only applicable to classifiers. I'm currently using regression!!
#print(classification_report(y_true, y_pred))

predictions = pd.DataFrame(y_pred, columns=["x", "y"])
result = y_true.reset_index(drop=True).join(predictions, rsuffix="_predicted")
result["error"] = (predictions[["x", "y"]] - result[["x", "y"]]).apply(np.linalg.norm, axis=1)
metrics = experiment_metrics(result)
pd.DataFrame([metrics], columns=list(metrics.keys()))

,mean_absolute_error,std_dev_distance_error,mean_squared_error,percentile_25,percentile_50,percentile_75,percentile_90,percentile_95,min,max
0,0.182199,0.641172,0.443476,0.0,0.0,1.776357e-15,0.899605,1.204022,0.0,6.0


## Parameter Sweeping ##

Initialize some variables with the values of each parameter that is going to be swept.

In [23]:
k_neighbors_values = range(1,10)
weights_values = ["uniform", "distance"]
metric_values = ["euclidean","manhattan", "chebyshev", "canberra", "braycurtis"]
nan_filler_values = [-100.0, -100000.0]
units_values = ["dBm", "mW"]
scaler_values = {"None": None,
                 "MinMaxScaler": preprocessing.MinMaxScaler(),
                 "StandardScaler": preprocessing.StandardScaler(),
                 "RobustScaler": preprocessing.RobustScaler(),
                 "NormalizerEuclidean": preprocessing.Normalizer(norm="l2"),
                 "NormalizerManhattan": preprocessing.Normalizer(norm="l1")}

#### Manual Parameter Estimation with for loops ####
Do the actual parameter estimation by manually sweeping through each parameter's possible value, while keeping track of the metrics for each parameter combination.

In [24]:
scenarios = []
scenario_keys = None
# for k_neighbors in k_neighbors_values:
#     for weights in weights_values:
#         for metric in metric_values:
#             for nan_filler in nan_filler_values:
#                 for units in units_values:
#                     for scaler_name, scaler in scaler_values.items():
#                         for data_scenario, data in data_scenarios.items():
#                             for test_data_scenario, test_data in test_data_scenarios.items():
#                                 if k_neighbors < len(data):
# #                                     print("train_data =", data_scenario)
# #                                     print("test_data =", test_data_scenario)
# #                                     print("train_data_size =", len(data))
# #                                     print("test_data_size =", len(test_data))
# #                                     print("algorithm =", "KNeighborsRegressor")
# #                                     print("n_neighbors =", k_neighbors)
# #                                     print("weights =", weights)
# #                                     print("metric =", metric)
# #                                     print("nan_filler =", nan_filler)
# #                                     print("units =", units)
# #                                     print("scaler =", scaler_name)
# #                                     print("----------------------------------------------------------------")
#                                     print(".", end='')
#                                     scenario = collections.OrderedDict([("train_data", data_scenario),
#                                                                         ("test_data", test_data_scenario),
#                                                                         ("train_data_size", len(data)),
#                                                                         ("test_data_size", len(test_data)),
#                                                                         ("algorithm", "KNeighborsRegressor"),
#                                                                         ("n_neighbors", k_neighbors),
#                                                                         ("weights", weights),
#                                                                         ("metric", metric),
#                                                                         ("nan_filler", nan_filler),
#                                                                         ("units", units),
#                                                                         ("scaler", scaler_name)])
#                                     curr_data = data.fillna(nan_filler)
#                                     curr_test_data = test_data.fillna(nan_filler)
#                                     curr_data[mac_addresses] = convert_to_units(curr_data[mac_addresses],
#                                                                                 from_units="dBm",
#                                                                                 to_units=units)
#                                     curr_test_data[mac_addresses] = convert_to_units(curr_test_data[mac_addresses],
#                                                                                      from_units="dBm",
#                                                                                      to_units=units)
#                                     if scaler is not None:
#                                         scaler.fit(curr_data[mac_addresses])
#                                         curr_data[mac_addresses] = pd.DataFrame(scaler.transform(curr_data[mac_addresses]),
#                                                                                 columns=mac_addresses)
#                                         curr_test_data[mac_addresses] = pd.DataFrame(scaler.transform(curr_test_data[mac_addresses]),
#                                                                                      columns=mac_addresses)
#                                     scenario.update(experiment_metrics(knn_experiment(curr_data,
#                                                                                       mac_addresses,
#                                                                                       ["x", "y"],
#                                                                                       algorithm="brute",
#                                                                                       n_neighbors=k_neighbors,
#                                                                                       weights=weights,
#                                                                                       metric=metric,
#                                                                                       test_data=curr_test_data)))
#                                     scenario_keys = scenario.keys()
#                                     scenarios.append(scenario)
print("\n"+str(len(scenarios))+" scenarios have been simulated.")


0 scenarios have been simulated.


Save the metrics to disk for further analysis.

In [25]:
metrics = pd.DataFrame(scenarios, columns=scenario_keys)
metrics.to_csv(output_data_directory + "/metrics.csv")